<a href="https://colab.research.google.com/github/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/blob/main/Try%20CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Try CatBoostRegressor

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

!pip install openpyxl -q
import openpyxl

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

# missing data
!pip install missingno -q
import missingno as msno

# for map
!pip install -q folium matplotlib
import folium
from matplotlib.colors import LinearSegmentedColormap, Normalize, to_hex

# for grid
!pip install geopy -q
from geopy.distance import geodesic

!pip install pandas numpy libpysal geopy esda -q
from geopy.distance import great_circle

# for Moran
!pip install pandas numpy libpysal geopy esda -q
import libpysal
from esda.moran import Moran

# linear regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Set display options
%matplotlib inline
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 1000)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 11.1 MB/s eta 0:00:00


In [5]:
apartments_initial = pd.read_excel('https://raw.githubusercontent.com/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/main/apartments4.xlsx', engine='openpyxl')

In [6]:
apartments = apartments_initial.copy()

In [7]:
import pickle
import requests
from io import BytesIO  # Import BytesIO

url_lat = 'https://raw.githubusercontent.com/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/main/lat_bin.pkl'
url_lon = 'https://raw.githubusercontent.com/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/main/lon_bin.pkl'

# Download lat_bin
response_lat = requests.get(url_lat)
lat_bin = pickle.load(BytesIO(response_lat.content))  # Load from the downloaded content

# Download lon_bin
response_lon = requests.get(url_lon)
lon_bin = pickle.load(BytesIO(response_lon.content))  # Load from the downloaded content

In [15]:
# удалить записи  если в ['Latitude_Bin', 'Longitude_Bin'] менее 3-х записей

# Calculate the number of listings in each bin
bin_counts = apartments.groupby(['Latitude_Bin', 'Longitude_Bin']).size().reset_index(name='count')

# Filter out bins with less than 4 listings
bins_to_keep = bin_counts[bin_counts['count'] >= 3][['Latitude_Bin', 'Longitude_Bin']]

# Merge with the original DataFrame to keep only listings in the selected bins
apartments1 = apartments.merge(bins_to_keep, on=['Latitude_Bin', 'Longitude_Bin'], how='inner')


In [8]:
apartments.columns

Index(['Listing ID', 'Host Since', 'Host Response Time', 'Host Response Rate',
       'Is Superhost', 'neighbourhood', 'Neighborhood Group', 'Postal Code',
       'Latitude', 'Longitude', 'Is Exact Location', 'Property Type',
       'Room Type', 'Accomodates', 'Bathrooms', 'Bedrooms', 'Beds', 'Price',
       'Guests Included', 'Min Nights', 'Reviews', 'First Review',
       'Last Review', 'Overall Rating', 'Accuracy Rating',
       'Cleanliness Rating', 'Checkin Rating', 'Communication Rating',
       'Location Rating', 'Value Rating', 'Instant Bookable',
       'Host Since Years', 'Last Review Years', 'First Review Years',
       'Postal Code1', 'Location Rating Roughly', 'Reviews Log', 'Price Log',
       'Neighborhood Location Rating', 'Neighborhood Price', 'Latitude_Bin',
       'Longitude_Bin'],
      dtype='object')

In [9]:
from sklearn.model_selection import train_test_split

X = apartments.drop('Overall Rating', axis=1)  # Признаки
y = apartments['Overall Rating']  # Целевая переменная

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
!pip install catboost -q
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from catboost import CatBoostRegressor, Pool  # Now CatBoost is available


# Предположим, что у вас есть датафрейм df, где 'target' - это ваш целевой признак
# числовые признаки в num_features и категориальные в cat_features

# Создадим пример датафрейма
data = {
    'num_feature1': np.random.rand(100),
    'num_feature2': np.random.rand(100),
    'cat_feature1': np.random.choice(['A', 'B', 'C'], 100),
    'cat_feature2': np.random.choice(['X', 'Y', 'Z'], 100),
    'target': np.random.rand(100)
}
df = pd.DataFrame(data)

# Определяем числовые и категориальные признаки
num_features = ['num_feature1', 'num_feature2']
cat_features = ['cat_feature1', 'cat_feature2']

# Определяем признаки и таргет
X = df[num_features + cat_features]
y = df['target']

# Разбиваем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

# Создаем модели для обучающей и тестовой выборок с указанием категориальных признаков
train_pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)

# Инициализируем и обучаем модель
model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, verbose=100)
model.fit(train_pool)

# Предсказания на тестовой выборке
y_pred = model.predict(test_pool)

# Вычисляем метрики
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Выводим метрики
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")


0:	learn: 0.2834352	total: 773us	remaining: 773ms
100:	learn: 0.1323910	total: 79.7ms	remaining: 710ms
200:	learn: 0.0640990	total: 170ms	remaining: 674ms
300:	learn: 0.0378190	total: 257ms	remaining: 596ms
400:	learn: 0.0251966	total: 345ms	remaining: 516ms
500:	learn: 0.0162980	total: 432ms	remaining: 431ms
600:	learn: 0.0113270	total: 534ms	remaining: 354ms
700:	learn: 0.0079562	total: 615ms	remaining: 262ms
800:	learn: 0.0053898	total: 774ms	remaining: 192ms
900:	learn: 0.0038131	total: 1.04s	remaining: 115ms
999:	learn: 0.0026881	total: 1.19s	remaining: 0us
Mean Squared Error: 0.10656106857351375
Mean Absolute Error: 0.2598731345859825
R^2 Score: -0.23135777623895892
